# Security Posture Workflow

A step by step example workflow to measure the security posture of a Kubernetes cluster using KubeHound.

## Initial Setup

Connect to the kubegraph server by running the cell below

In [ ]:
%%graph_notebook_config
{
  "host": "host.docker.internal",
  "port": 8182,
  "ssl": false,
  "gremlin": {
    "traversal_source": "g",
    "username": "",
    "password": "",
    "message_serializer": "graphsonv3"
  }
}

## Workflow

### High Level Metrics

Let us get a high-level view of the security posture of the cluster. These metrics are not very nuanced but provide a top-level view of cluster security and easy tracking of improvements over time.

First let's look at the shortest path from external service to a critical asset

In [ ]:
%%gremlin
kh.services().minHopsToCritical()

Next let's see the total number of attacks paths originating from external services

In [ ]:
%%gremlin
kh.services().criticalPaths().count()

### Exposed asset analyis

The most likely entry points for an attacker into a Kubernetes cluster are:
+ Exposed services via 0day, n-day, or misconfigurations
+ Leaked credentials
+ Supply chain attacks leading to execution within a container

We can use KubeHound to evaluate the percentage of each of these entry points that can lead to a critical asset. First services:

In [ ]:
%%gremlin
kh.V().
    hasLabel("Endpoint").
    has("exposure", gte(2)).    // https://kubehound.io/queries/dsl/#endpoint-exposure
    count().
    aggregate("t").
    V().
    hasLabel("Endpoint").
    has("exposure", gte(2)).    // https://kubehound.io/queries/dsl/#endpoint-exposure
    hasCriticalPath().
    count().
    as("e").
    math("100 * e/t").by().by(unfold())

Next credentials:

In [ ]:
%%gremlin
kh.V().
    hasLabel("Identity").
    has("critical", false).
    count().
    aggregate("t").
    V().
    hasLabel("Identity").
    has("critical", false).
    hasCriticalPath().
    count().
    as("e").
    math("100 * e/t").by().by(unfold())

Finally containers:

In [ ]:
%%gremlin
kh.V().
    hasLabel("Container").
    count().
    aggregate("t").
    V().
    hasLabel("Container").
    hasCriticalPath().
    count().
    as("e").
    math("100 * e/t").by().by(unfold())

### Threat Modelling

KubeHound can provide a high level overview of attack paths grouped by frequency in any given cluster via the DSL

In [ ]:
%%gremlin
kh.services().criticalPathsFreq()